In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import plotly.express as px

import missingno as msno
import re
from uszipcode import SearchEngine, SimpleZipcode, ComprehensiveZipcode

import seaborn as sns
sns.set(rc={'figure.figsize':(14.15,10)})
import matplotlib.pyplot as plt

import warnings
# warnings.filterwarnings('ignore')

In [2]:
%load_ext rpy2.ipython
# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

<rpy2.rinterface_lib.sexp.NULLType object at 0x7fec5db0b940> [RTYPES.NILSXP]

In [3]:
# R package names
packnames = ('olsrr', 'car', 'corrplot')

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

In [4]:
%%R -o pairsJDRS -o myResPlots -o myResPlots2
library(olsrr)
library(car)

# Use: Best Subsets Regression
# library(leaps)
library(corrplot)

# Use: Plot CIs
# library(plotrix)

pairsJDRS <- function (R, ...) {
  panel.cor <- function(x, y, digits = 2, prefix = "", ...) {
    usr <- par("usr")
    on.exit(par(usr))
    par(usr = c(0, 1, 0, 1))
    r <- cor(x, y, use = "pairwise.complete.obs", method = "pearson")
    txt <- format(c(r, 0.123456789), digits = digits)[1]
    txt <- paste(prefix, txt, sep = "")
    cex <- 0.8/strwidth(txt)
    test <- cor.test(x, y)
    Signif <- symnum(test$p.value, corr = FALSE, na = FALSE, 
                     cutpoints = c(0, 0.001, 0.01, 0.05, 0.1, 1), symbols = c("***", 
                                                                              "**", "*", ".", " "))
    text(0.5, 0.5, txt, cex = cex * (abs(r) + 0.3)/1.3)
    text(0.8, 0.8, Signif, cex = cex, col = 2)
  }
  hist.panel = function(x, ...) {
    par(new = TRUE)
    hist(x, col = "light gray", probability = TRUE, axes = FALSE, 
         main = "", breaks = "FD")
    lines(density(x, na.rm = TRUE), col = "red", lwd = 1)
    rug(x)
  }
  pairs(R, gap = 0, upper.panel = panel.smooth, lower.panel = panel.cor, 
        diag.panel = hist.panel,  ...)
}


myResPlots <- function(model, label){
  
  #Normal quantile plot of studentized residuals
  qqPlot(rstudent(model), pch=19, main=paste("NQ Plot of Studentized Residuals,",label))
  
  #plot of fitted vs. studentized residuals
  plot(rstudent(model) ~ model$fitted.values, pch=19, col='red', xlab="Fitted Values", ylab = "Studentized Residuals",
     main = paste("Fits vs. Studentized Residuals,", label))
  abline(h=0, lwd=3)
  abline(h=c(3,-3), lty=2, lwd=3, col="blue")

  #Cooks distance plot
  ols_cooksd_chart(model)
  
  #Combination outlier and influence plot
  ols_rsdlev_plot(model)
}

myResPlots2 <- function(model, label = "Residual Plots"){
  
  #Normal quantile plot of studentized residuals
  qqPlot(rstudent(model), pch=19, main=paste("NQ Plot of Studentized Residuals,",label))
  
  #plot of fitted vs. studentized residuals
  plot(rstudent(model) ~ model$fitted.values, pch=19, col='red', xlab="Fitted Values", ylab = "Studentized Residuals",
     main = paste("Fits vs. Studentized Residuals,", label))
  abline(h=0, lwd=3)
  abline(h=c(3,-3), lty=2, col="green")
  abline(h=c(2,-2), lty=2, col="blue")

}

R[write to console]: 
Attaching package: ‘olsrr’


R[write to console]: The following object is masked from ‘package:datasets’:

    rivers


R[write to console]: corrplot 0.92 loaded



In [33]:
# Reading data
accepted = pd.read_csv('data/LC/Lending_Club_Accepted_2014_2018.csv')
rejected = pd.read_csv('data/LC/Lending_Club_Rejected_2014_2018.csv')
print(accepted.shape)
display(accepted.head())
print(rejected.shape)
display(rejected.head())

/usr/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3397: DtypeWarning: Columns (19,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(2029952, 151)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


(26132308, 9)


,Amount_Requested,Application_Date,Loan_Title,Risk_Score,Debt_to_Income_Ratio,Zip_Code,State,Employment_Length,Policy_Code
0,1000.0,2016-04-01,other,NaN,2.69%,331xx,FL,< 1 year,0.0
1,4000.0,2016-04-01,debt_consolidation,NaN,28.26%,834xx,ID,< 1 year,0.0
2,5000.0,2016-04-01,moving,NaN,-1%,648xx,MO,NaN,0.0
3,1000.0,2016-04-01,moving,628.0,21.43%,380xx,TN,< 1 year,0.0
4,3000.0,2016-04-01,Debt consolidation,NaN,8.49%,895xx,NV,2 years,2.0


In [34]:
stripped_accepted = accepted[['loan_amnt', 'issue_d', 'title', 'dti', 'zip_code', 'addr_state', 'emp_length', 'policy_code']]
stripped_accepted['accepted'] = 1
rejected.rename(columns={'Amount_Requested': 'loan_amnt',
                         'Application_Date': 'date',
                         'Loan_Title': 'title',
                         'Risk_Score': 'risk_score',
                         'Debt_to_Income_Ratio': 'dti',
                         'Zip_Code': 'zip_code',
                         'State': 'state',
                         'Employment_Length': 'emp_length',
                         'Policy_Code': 'policy_code'}, inplace=True)
rejected.drop(columns='risk_score', inplace=True)
rejected['accepted'] = 0
stripped_accepted.rename(columns={'issue_d': 'date',
                                  'addr_state': 'state'}, inplace=True)
print("Accepted")
for col in stripped_accepted.columns:
    prop_missing = stripped_accepted[col].isna().mean()
    print(f'Column: {col}, Proportion missing: {prop_missing}')
print("Rejected")
for col in rejected.columns:
    prop_missing = rejected[col].isna().mean()
    print(f'Column: {col}, Proportion missing: {prop_missing}')

/tmp/ipykernel_2404728/881687582.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stripped_accepted['accepted'] = 1
/usr/lib64/python3.10/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Accepted
Column: loan_amnt, Proportion missing: 0.0
Column: date, Proportion missing: 0.0
Column: title, Proportion missing: 0.011480567028185888
Column: dti, Proportion missing: 0.0008428770729554196
Column: zip_code, Proportion missing: 4.926224856548332e-07
Column: state, Proportion missing: 0.0
Column: emp_length, Proportion missing: 0.06793658175168674
Column: policy_code, Proportion missing: 0.0
Column: accepted, Proportion missing: 0.0
Rejected
Column: loan_amnt, Proportion missing: 0.0
Column: date, Proportion missing: 0.0
Column: title, Proportion missing: 4.932591487900724e-05
Column: dti, Proportion missing: 0.0
Column: zip_code, Proportion missing: 1.0370304834919289e-05
Column: state, Proportion missing: 3.826680750892726e-08
Column: emp_length, Proportion missing: 0.03566757287569089
Column: policy_code, Proportion missing: 3.512892929319523e-05
Column: accepted, Proportion missing: 0.0


In [15]:
loan_titles = ['Debt Consolidation', 'Credit Card', 'Home', 'Car', 'Medical', 'Business', 'Other']

In [35]:
full_data = pd.concat([stripped_accepted, rejected]).reset_index().drop(columns='index')
print(full_data.head())
print(full_data.tail())

   loan_amnt      date               title    dti zip_code state emp_length  \
0     3600.0  Dec-2015  Debt consolidation   5.91    190xx    PA  10+ years   
1    24700.0  Dec-2015            Business  16.06    577xx    SD  10+ years   
2    20000.0  Dec-2015                 NaN  10.78    605xx    IL  10+ years   
3    35000.0  Dec-2015  Debt consolidation  17.06    076xx    NJ  10+ years   
4    10400.0  Dec-2015      Major purchase  25.37    174xx    PA    3 years   

   policy_code  accepted  
0          1.0         1  
1          1.0         1  
2          1.0         1  
3          1.0         1  
4          1.0         1  
          loan_amnt        date               title     dti zip_code state  \
28162255    10000.0  2016-12-31  Debt consolidation  41.26%    441xx    OH   
28162256    10000.0  2016-12-31              moving   1.48%    207xx    MD   
28162257     1200.0  2016-12-31               Other  10.26%    914xx    CA   
28162258    25000.0  2016-12-31  debt_consolidation

In [36]:
counter = 0
def clean_titles(full_data):
    full_data['title'] = full_data['title'].fillna('')
    full_data['title'] = full_data['title'].str.lower()
    full_data.loc[full_data['title'].str.contains('credit'), 'title'] = 'Credit Card'
    full_data.loc[full_data['title'].str.contains('cc'), 'title'] = 'Credit Card'
    full_data.loc[full_data['title'].str.contains('debt'), 'title'] = 'Debt Consolidation'
    full_data.loc[full_data['title'].str.contains('consolidation'), 'title'] = 'Debt Consolidation'
    full_data.loc[full_data['title'].str.contains('consolidate'), 'title'] = 'Debt Consolidation'
    full_data.loc[full_data['title'].str.contains('payoff'), 'title'] = 'Debt Consolidation'
    full_data.loc[full_data['title'].str.contains('pay off'), 'title'] = 'Debt Consolidation'
    full_data.loc[full_data['title'].str.contains('refinance'), 'title'] = 'Debt Consolidation'
    full_data.loc[full_data['title'].str.contains('home'), 'title'] = 'Home'
    full_data.loc[full_data['title'].str.contains('house'), 'title'] = 'Home'
    full_data.loc[full_data['title'].str.contains('moving'), 'title'] = 'Home'
    full_data.loc[full_data['title'].str.contains('car'), 'title'] = 'Car'
    full_data.loc[full_data['title'].str.contains('medical'), 'title'] = 'Medical'
    full_data.loc[full_data['title'].str.contains('business'), 'title'] = 'Business'
    full_data.loc[~full_data['title'].isin(loan_titles), 'title'] = 'Other'
    return full_data

def clean_date(data):
    data['date'] = pd.to_datetime(data['date']).dt.strftime('%m-%Y')
    return data
    
def clean_dti(data):
    # Remove all rows with negative dti
    data['dti'] = data['dti'].astype(str)
    data = data[~data['dti'].str.contains('-')]
    
    # Replace %
    data['dti'] = data['dti'].str.replace('%', '').astype('float64')
    data = data[~data['dti'].isna()]
    
    return data
    
def clean_zip(data):
    # Remove na values first or else error
    data = data[~data['zip_code'].isna()]
    data = data[~data['state'].isna()]
    
    # Get first 3 digits
    data['zip_code'] = data['zip_code'].apply(lambda x: x[:3])

    # Remove the one case where zipcode does not begin with 3 digits
    data = data[data['zip_code'].apply(str.isdigit)]    
    
    # Make the mapping zip -> state code
    search = SearchEngine()
    zips = set()
    zip2state = {}
    for zip_code in data['zip_code']:
        zips.add(zip_code)
    for zip_code in zips:
        res = search.by_prefix(zip_code)
        if res == []:
            zip2state[zip_code] = ''
        else:
            zip2state[zip_code] = res[0].state
            
    # Use knowledge of correct zip_codes to make correct state -> zipcode
    def matchStateZip(zip_code, state):
        global counter
        counter += 1
        if counter % 100000 == 0:
            print(counter)
        return state == zip2state[zip_code]

    with SearchEngine() as search:
        to_remove = data.apply(lambda x: matchStateZip(x.zip_code, x.state), axis=1)

    print(f'Shape before {data.shape}')
    data = data[to_remove]
    print(f'Shape after removing incorrect state <-> zip code matchings {data.shape}')
    
    return data

def clean_full(data):
    # loan_amnt is clean
    # Clean titles first
    print('clean titles')
    data = clean_titles(data)
    
    # Clean date
    print('clean date')
    data = clean_date(data)
    
    # Clean dti
    print('clean dti')
    data = clean_dti(data)
    
    # Clean zip + state combo
    print('clean zip')
    data = clean_zip(data)
    
    # # policy code corresponds with being accepted/rejected except for special policy code 2.0
    # ALL POLICY CODE 2.0 (https://news.fintechnexus.com/policy-code-2-loans-lending-club/) LOANS WERE REJECTED APPS
    print('clean policy code')
    data = data.drop(columns='policy_code')
    
    # Keep emp_length the same (will drop na values)
    
    # Remove NA Last
    print('clean na')
    prev_rows = data.shape[0]
    data = data.dropna()
    print(f'Training examples reduced by {1-(data.shape[0]/prev_rows)}')
    return data.reset_index().drop(columns='index')

full_data = clean_full(full_data)

clean titles
clean date
clean dti


/tmp/ipykernel_2404728/3913760416.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['dti'] = data['dti'].str.replace('%', '').astype('float64')


clean zip
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000


In [49]:
full_data

,loan_amnt,title,dti,zip_code,state,emp_length,accepted,date
0,3600.0,Debt Consolidation,5.91,190,PA,10+ years,1,12-2015
1,24700.0,Business,16.06,577,SD,10+ years,1,12-2015
2,20000.0,Other,10.78,605,IL,10+ years,1,12-2015
3,35000.0,Debt Consolidation,17.06,076,NJ,10+ years,1,12-2015
4,10400.0,Other,25.37,174,PA,3 years,1,12-2015
...,...,...,...,...,...,...,...,...
25871613,10000.0,Debt Consolidation,41.26,441,OH,< 1 year,0,12-2016
25871614,10000.0,Home,1.48,207,MD,5 years,0,12-2016
25871615,1200.0,Other,10.26,914,CA,< 1 year,0,12-2016
25871616,25000.0,Debt Consolidation,17.71,880,NM,< 1 year,0,12-2016


In [66]:
from shapely.geometry import Polygon, MultiPolygon
from collections import Counter

In [80]:
# Append the data with zipcode information
    
# Rural definition (https://www.ers.usda.gov/topics/rural-economy-population/rural-classifications/what-is-rural.aspx#:~:text=According%20to%20this%20system%2C%20rural,with%20fewer%20than%202%2C500%20people.)
# def add_zip_info(data):
# search2 = SearchEngine(simple_or_comprehensive=SearchEngine.SimpleOrComprehensiveArgEnum.comprehensive)
# zips = set()
# zip2data = {}
# for zip_code in data['zip_code']:
#     zips.add(zip_code)
# for zip_code in zips:
res = search2.by_prefix("405", returns=0)
if res == []:
    zip2state[zip_code] = pd.DataFrame({'population': pd.NA, 'population_density': pd.NA, 'median_home_value': pd.NA, 'median_household_income': pd.NA, 'polygon': pd.NA, 'population_by_gender': pd.NA, 'population_by_race': pd.NA}, index=[0])
else:
    # Get consolidate polygon, gender, race
    polygons = []
    genders = Counter()
    races = Counter()
    for zipcode in res:
        # Multipolygon
        if zipcode.polygon is not None:
            if len(zipcode.polygon) == 1:
                polygons.append(Polygon(zipcode.polygon[0]))
            else:
                polygons.append(Polygon(zipcode.polygon))
        # Gender
        if zipcode.population_by_gender is not None:
            gender = zipcode.population_by_gender[0]['values']
            gender = Counter({gender[0]['x']: gender[0]['y'], gender[1]['x']: gender[1]['y']})
            print(sum(gender.values()))
            print(zipcode.population)
            assert(sum(gender.values()) == zipcode.population)
            
            genders += gender
        # Race
        if zipcode.population_by_race is not None:
            race = zipcode.population_by_race[0]['values']
            for item in race:
                races += Counter({item['x']: item['y']})
                
    mp = MultiPolygon(polygons)
    
    # Get gender dict
    

    # Get other information
    print(pd.DataFrame({'population': zipcode.population, 
                                        'population_density': zipcode.population_density, 
                                        'median_home_value': zipcode.median_home_value, 
                                        'median_household_income': zipcode.median_household_income, 
                                        'polygon': mp, 
                                        'pop_male': genders['Male'], 
                                        'pop_female': genders['Female']}))
    zip2state[zip_code] = {'population': zipcode.population, 
                            'population_density': zipcode.population_density, 
                            'median_home_value': zipcode.median_home_value, 
                            'median_household_income': zipcode.median_household_income, 
                            'polygon': mp, 
                            'population_by_gender': dict(genders), 
                            'population_by_race': dict(races)}

# User the zip2data to append everything but polygon

#     return data
    


25709
25709
28003
28003
25655
25655
26040
26040
1757
1757
23569
23569
32364
32364
1928
1928
31798
31798
10994
10994
14647
14647
33595
33595
2877
2877
35227
35227
    population  population_density  median_home_value  \
0        35227              5705.0             128700   
1        35227              5705.0             128700   
2        35227              5705.0             128700   
3        35227              5705.0             128700   
4        35227              5705.0             128700   
5        35227              5705.0             128700   
6        35227              5705.0             128700   
7        35227              5705.0             128700   
8        35227              5705.0             128700   
9        35227              5705.0             128700   
10       35227              5705.0             128700   
11       35227              5705.0             128700   
12       35227              5705.0             128700   
13       35227              5705.0      

/usr/lib64/python3.10/site-packages/pandas/core/internals/construction.py:622: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  raw_lengths.append(len(val))


NameError: name 'zip2state' is not defined